In [44]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [45]:
df = pd.read_csv('Advertising.csv')
df

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


## Разбиваем данные на обучающий и тестовый наборы данных - Train | Test Split

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Применяем масштабирование (scale) для тестовых данных X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на тестовых данных X (создавая предсказания и сравнивая их с Y_test)
7. Уточняем параметры модели, повторяя шаги 5 и 6

In [46]:
df.isnull().sum() # отсутствующих данных нет, поэтому создаём признаки и целевую переменную

TV           0
radio        0
newspaper    0
sales        0
dtype: int64

In [47]:
X = df.drop('sales', axis=1)

In [48]:
y = df['sales']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [50]:
# нужно провести масштабирование т.к. разные признаки могут быть представлены в разных единицах

In [51]:
scaler = StandardScaler() 

In [52]:
scaler.fit(X_train) # тестовый набор нельзя, иначе утечка данных

StandardScaler()

In [53]:
# промасштабировали признаки
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
model = Ridge(alpha=100)

In [55]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [56]:
y_pred = model.predict(X_test)

In [57]:
# оценка модели
mean_squared_error(y_test, y_pred)

7.341775789034128

In [58]:
# настройка гиперпараметров
optimal_error = 10000
alpha_optimal = -100
for alpha_i in np.linspace(0, 1, 100):
    model = Ridge(alpha=alpha_i)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    error = mean_squared_error(y_test, y_pred)
    if error < optimal_error:
        alpha_optimal = alpha_i
        optimal_error = error

In [59]:
print(optimal_error, alpha_optimal)

2.2987166978863796 0.0


In [60]:
# Есть минус такого подхода - в цикле нужно искать оптимальное решение. Существует иной подход, в котором будет поиск по сетке.

In [61]:
#Train|Test|Split

----
## Обучение - валидация - тестирование (Train | Validation | Test Split Procedure)

Этот подход ещё называют методом с задержкой ("hold-out"), потому что здесь мы не настраиваем параметры модели на финальном тестовом наборе данных, а используем тестовые данные *только* для оценки финального результата.

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий, оценочный и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Масштабируем (scale) оценочные данные X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на оценочных данных X (создавая предсказания и сравнивая их с Y_eval)
7. Уточняем параметры модели, повторяя шаги 5 и 6
8. Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)

In [62]:
X = df.drop('sales', axis=1)

In [63]:
y = df['sales']

In [64]:
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=101)

In [65]:
# test_size = 0.5 ( 50% от 30% other ---> test_size - 15 % от всего объема данных)
# То, какой процент брать, зависит от объема данных. Если миллион строк, то стоит их уменьшить

X_eval, X_test, y_eval, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=101)

In [66]:
scaler1 = StandardScaler()

In [67]:
scaler1.fit(X_train)

StandardScaler()

In [68]:
# масштабирование
X_train = scaler1.transform(X_train)
X_eval = scaler1.transform(X_eval)
X_test = scaler1.transform(X_test)

In [69]:
model_one = Ridge(alpha=100)

In [70]:
model_one.fit(X_train, y_train)

Ridge(alpha=100)

In [71]:
y_eval_pred = model_one.predict(X_eval)

In [72]:
mean_squared_error(y_eval, y_eval_pred)

7.320101458823869

In [73]:
# настройка гиперпараметров
optimal_error1 = 10000
alpha_optimal1 = -100
model = None
for alpha_i in range(1, 100):
    model_m = Ridge(alpha=alpha_i)
    model_m.fit(X_train, y_train)
    y_eval_pred = model_m.predict(X_eval)
    error = mean_squared_error(y_eval, y_eval_pred)
    if error < optimal_error1:
        model = model_m
        alpha_optimal1 = alpha_i
        optimal_error1 = error

In [74]:
print(optimal_error1, alpha_optimal1)

2.3837830750569853 1


In [75]:
# Финальная проверка на тест
# Выбираем лучшую модель

In [76]:
y_final_test_pred = model.predict(X_test)

In [77]:
mean_squared_error(y_test, y_final_test_pred)

2.254260083800517

----
## Кросс-валидация с помощью cross_val_score

----

<img src="grid_search_cross_validation.png" alt="Схема" width="900" height="500">

----

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [79]:
scaler.fit(X_train)

StandardScaler()

In [80]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [81]:
model = Ridge(alpha=1)

In [82]:
#estimator - используемая модель для оценки
scores = cross_val_score(model, X_train, y_train, scoring ='neg_mean_squared_error', cv=5)
# cv - число перекресных проверок

In [83]:
abs(scores.mean())

np.float64(3.344839296530695)

In [84]:
# далее настройка гиперпараметров
# поиск оптимального альфа
# если поиск выполнен, то дальше начинаем тренировать модель

In [85]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [86]:
y_final_test_pred = model.predict(X_test)

In [87]:
mean_squared_error(y_test, y_final_test_pred)

2.3190215794287514